In [8]:
import serial

import time
import csv
import matplotlib
matplotlib.use("tkAgg")
import matplotlib.pyplot as plt
import numpy as np

import re
import pandas as pd
import dask.dataframe as dd
import streamz.dataframe as sdf
# from streamz.dataframe import StreamingDataFrame

# import threading, queue
import multiprocess as mp
# from multiprocessing import Process, Pipe
import signal
import os
from subprocess import check_output

In [9]:
portAddr = "/dev/cu.usbmodem14101"
baud = 115200

In [10]:
ser = serial.Serial(
    port=portAddr,
    baudrate=baud,
    bytesize=8,
    timeout=2,
    stopbits=serial.STOPBITS_ONE
)
ser.flushInput()



In [11]:
# plot_window = 20
# y_var = np.array(np.zeros([plot_window]))

# plt.ion()
# fig, ax = plt.subplots()
# line, = ax.plot(y_var)


In [12]:
def process_serial_out(ascii_string):
    ascii_string += ','
    findall = re.findall(r'(?P<var>.*?):(?P<out>.*?),', ascii_string)
    extracted = {k.strip(): v.strip() for k,v in findall}
    return extracted

In [13]:

def arduino_worker(queue):
    print("worker starting...")
    while True:
        try:
            ser_bytes = ser.readline()
            try:
                print("got", ser_bytes)
                decoded_bytes = ser_bytes.decode('ascii')
                row = process_serial_out(decoded_bytes)
                print(decoded_bytes)
                queue.put(row)
            except:
                print_exc()
                print("exception reading one line from arduino")
                continue
        except:
            print_exc()
            print("Keyboard Interrupt")
            break

In [14]:
import random

def stride_boundary(row, this_stride):
#     just a dummy code. We need to implement the logic.
    if (bool(random.getrandbits(1))):
        return True
    else:
        return False

In [15]:


#worker to process one stride boundary, sort data into buckets
def stride_processor_worker(total_data, raw_stream):
    this_stride = []
    print("stride_processor starting...")
    while True:
        #Infinitely 1. appends to this_stride, then when boundary is met,
        #appends that stride to total_data, reinitializes this_stride
        #and loops forever.
        try:
            row = raw_stream.get()
            print ("decoded: ", row)
            if len(this_stride) > 5:
                #put away the current stride in total data as one stride
                total_data.put(this_stride)
                print("finished writing this one stride, size: ", len(this_stride))
                #reinitialize the stride queue
                this_stride = []
            else:
                this_stride.append(row)
        except:
            print_exc()
            print("exception in stride processor worker")
            
    

In [16]:
# overall structure:
# total_data = [stride1, stride2,,,,,,stride500]
# stride1 = [{r1}, {r2}, {r3}]

In [17]:
######### start 'main' ########

In [18]:
pool = mp.Pool()
manager = mp.Manager()
usb_reading_queue = mp.SimpleQueue()
total_data = mp.SimpleQueue()

In [19]:

#  set up worker threads
# arduino_thread = threading.Thread(target=arduino_worker, args=[usb_reading_queue], daemon=True)

In [20]:
# stride_thread = threading.Thread(target=stride_processor_worker, args=[total_data], daemon=True)
stride_process = mp.Process(target=stride_processor_worker, args=[total_data, usb_reading_queue])
# stride_process = mp.Process(target=stride_processor_worker, args=(total_data, usb_reading_queue))


In [21]:
# arduino_res = pool.apply_async(arduino_worker, (usb_reading_queue))
# arduino_proc = mp.Process(target=arduino_worker, args=[usb_reading_queue])
arduino_proc = mp.Process(target=arduino_worker, args=[usb_reading_queue])


In [22]:
arduino_proc.start()

worker starting...
got b',Gyro_x: 0.72,Gyro_y: 0.69,Gyro_z: -0.05,Mag_x: 0.72,Magl_y: 0.69,Mag_z: -0.05\r\n'
,Gyro_x: 0.72,Gyro_y: 0.69,Gyro_z: -0.05,Mag_x: 0.72,Magl_y: 0.69,Mag_z: -0.05

got b'Time:5491612,Pitch:5.94,Yaw:88.17,Roll:-177.63,Accel_x: 0.72,Accel_y: 0.69,Accel_z: -0.05,Gyro_x: 0.72,Gyro_y: 0.69,Gyro_z: -0.05,Mag_x: 0.72,Magl_y: 0.69,Mag_z: -0.05\r\n'
Time:5491612,Pitch:5.94,Yaw:88.17,Roll:-177.63,Accel_x: 0.72,Accel_y: 0.69,Accel_z: -0.05,Gyro_x: 0.72,Gyro_y: 0.69,Gyro_z: -0.05,Mag_x: 0.72,Magl_y: 0.69,Mag_z: -0.05

got b'Time:5491619,Pitch:5.94,Yaw:88.17,Roll:-177.63,Accel_x: 0.72,Accel_y: 0.69,Accel_z: -0.05,Gyro_x: 0.72,Gyro_y: 0.69,Gyro_z: -0.05,Mag_x: 0.72,Magl_y: 0.69,Mag_z: -0.05\r\n'
Time:5491619,Pitch:5.94,Yaw:88.17,Roll:-177.63,Accel_x: 0.72,Accel_y: 0.69,Accel_z: -0.05,Gyro_x: 0.72,Gyro_y: 0.69,Gyro_z: -0.05,Mag_x: 0.72,Magl_y: 0.69,Mag_z: -0.05

got b'Time:5491626,Pitch:5.94,Yaw:88.17,Roll:-177.63,Accel_x: 0.72,Accel_y: 0.69,Accel_z: -0.05,Gyro_x: 0.72,Gyro_y

got b'Time:5491775,Pitch:5.94,Yaw:88.17,Roll:-177.63,Accel_x: 0.72,Accel_y: 0.69,Accel_z: -0.05,Gyro_x: 0.72,Gyro_y: 0.69,Gyro_z: -0.05,Mag_x: 0.72,Magl_y: 0.69,Mag_z: -0.05\r\n'
Time:5491775,Pitch:5.94,Yaw:88.17,Roll:-177.63,Accel_x: 0.72,Accel_y: 0.69,Accel_z: -0.05,Gyro_x: 0.72,Gyro_y: 0.69,Gyro_z: -0.05,Mag_x: 0.72,Magl_y: 0.69,Mag_z: -0.05

got b'Time:5491782,Pitch:5.94,Yaw:88.17,Roll:-177.63,Accel_x: 0.72,Accel_y: 0.69,Accel_z: -0.05,Gyro_x: 0.72,Gyro_y: 0.69,Gyro_z: -0.05,Mag_x: 0.72,Magl_y: 0.69,Mag_z: -0.05\r\n'
Time:5491782,Pitch:5.94,Yaw:88.17,Roll:-177.63,Accel_x: 0.72,Accel_y: 0.69,Accel_z: -0.05,Gyro_x: 0.72,Gyro_y: 0.69,Gyro_z: -0.05,Mag_x: 0.72,Magl_y: 0.69,Mag_z: -0.05

got b'Time:5491790,Pitch:5.94,Yaw:88.17,Roll:-177.63,Accel_x: 0.72,Accel_y: 0.69,Accel_z: -0.05,Gyro_x: 0.72,Gyro_y: 0.69,Gyro_z: -0.05,Mag_x: 0.72,Magl_y: 0.69,Mag_z: -0.05\r\n'
Time:5491790,Pitch:5.94,Yaw:88.17,Roll:-177.63,Accel_x: 0.72,Accel_y: 0.69,Accel_z: -0.05,Gyro_x: 0.72,Gyro_y: 0.69,Gyro_z: -


got b'Time:5491943,Pitch:5.94,Yaw:88.17,Roll:-177.63,Accel_x: 0.72,Accel_y: 0.69,Accel_z: -0.05,Gyro_x: 0.72,Gyro_y: 0.69,Gyro_z: -0.05,Mag_x: 0.72,Magl_y: 0.69,Mag_z: -0.05\r\n'
Time:5491943,Pitch:5.94,Yaw:88.17,Roll:-177.63,Accel_x: 0.72,Accel_y: 0.69,Accel_z: -0.05,Gyro_x: 0.72,Gyro_y: 0.69,Gyro_z: -0.05,Mag_x: 0.72,Magl_y: 0.69,Mag_z: -0.05

got b'Time:5491950,Pitch:5.94,Yaw:88.17,Roll:-177.63,Accel_x: 0.72,Accel_y: 0.69,Accel_z: -0.05,Gyro_x: 0.72,Gyro_y: 0.69,Gyro_z: -0.05,Mag_x: 0.72,Magl_y: 0.69,Mag_z: -0.05\r\n'
Time:5491950,Pitch:5.94,Yaw:88.17,Roll:-177.63,Accel_x: 0.72,Accel_y: 0.69,Accel_z: -0.05,Gyro_x: 0.72,Gyro_y: 0.69,Gyro_z: -0.05,Mag_x: 0.72,Magl_y: 0.69,Mag_z: -0.05

got b'Time:5491957,Pitch:5.94,Yaw:88.17,Roll:-177.63,Accel_x: 0.72,Accel_y: 0.69,Accel_z: -0.05,Gyro_x: 0.72,Gyro_y: 0.69,Gyro_z: -0.05,Mag_x: 0.72,Magl_y: 0.69,Mag_z: -0.05\r\n'
Time:5491957,Pitch:5.94,Yaw:88.17,Roll:-177.63,Accel_x: 0.72,Accel_y: 0.69,Accel_z: -0.05,Gyro_x: 0.72,Gyro_y: 0.69,Gyro_z: 


got b'Time:5492112,Pitch:5.94,Yaw:88.17,Roll:-177.63,Accel_x: 0.72,Accel_y: 0.69,Accel_z: -0.05,Gyro_x: 0.72,Gyro_y: 0.69,Gyro_z: -0.05,Mag_x: 0.72,Magl_y: 0.69,Mag_z: -0.05\r\n'
Time:5492112,Pitch:5.94,Yaw:88.17,Roll:-177.63,Accel_x: 0.72,Accel_y: 0.69,Accel_z: -0.05,Gyro_x: 0.72,Gyro_y: 0.69,Gyro_z: -0.05,Mag_x: 0.72,Magl_y: 0.69,Mag_z: -0.05

got b'Time:5492120,Pitch:5.94,Yaw:88.17,Roll:-177.63,Accel_x: 0.72,Accel_y: 0.69,Accel_z: -0.05,Gyro_x: 0.72,Gyro_y: 0.69,Gyro_z: -0.05,Mag_x: 0.72,Magl_y: 0.69,Mag_z: -0.05\r\n'
Time:5492120,Pitch:5.94,Yaw:88.17,Roll:-177.63,Accel_x: 0.72,Accel_y: 0.69,Accel_z: -0.05,Gyro_x: 0.72,Gyro_y: 0.69,Gyro_z: -0.05,Mag_x: 0.72,Magl_y: 0.69,Mag_z: -0.05

got b'Time:5492128,Pitch:5.94,Yaw:88.17,Roll:-177.63,Accel_x: 0.72,Accel_y: 0.69,Accel_z: -0.05,Gyro_x: 0.72,Gyro_y: 0.69,Gyro_z: -0.05,Mag_x: 0.72,Magl_y: 0.69,Mag_z: -0.05\r\n'
Time:5492128,Pitch:5.94,Yaw:88.17,Roll:-177.63,Accel_x: 0.72,Accel_y: 0.69,Accel_z: -0.05,Gyro_x: 0.72,Gyro_y: 0.69,Gyro_z: 


got b'Time:5492278,Pitch:5.94,Yaw:88.17,Roll:-177.63,Accel_x: 0.72,Accel_y: 0.69,Accel_z: -0.05,Gyro_x: 0.72,Gyro_y: 0.69,Gyro_z: -0.05,Mag_x: 0.72,Magl_y: 0.69,Mag_z: -0.05\r\n'
Time:5492278,Pitch:5.94,Yaw:88.17,Roll:-177.63,Accel_x: 0.72,Accel_y: 0.69,Accel_z: -0.05,Gyro_x: 0.72,Gyro_y: 0.69,Gyro_z: -0.05,Mag_x: 0.72,Magl_y: 0.69,Mag_z: -0.05

got b'Time:5492286,Pitch:5.94,Yaw:88.17,Roll:-177.63,Accel_x: 0.72,Accel_y: 0.69,Accel_z: -0.05,Gyro_x: 0.72,Gyro_y: 0.69,Gyro_z: -0.05,Mag_x: 0.72,Magl_y: 0.69,Mag_z: -0.05\r\n'
Time:5492286,Pitch:5.94,Yaw:88.17,Roll:-177.63,Accel_x: 0.72,Accel_y: 0.69,Accel_z: -0.05,Gyro_x: 0.72,Gyro_y: 0.69,Gyro_z: -0.05,Mag_x: 0.72,Magl_y: 0.69,Mag_z: -0.05

got b'Time:5492293,Pitch:5.94,Yaw:88.17,Roll:-177.63,Accel_x: 0.72,Accel_y: 0.69,Accel_z: -0.05,Gyro_x: 0.72,Gyro_y: 0.69,Gyro_z: -0.05,Mag_x: 0.72,Magl_y: 0.69,Mag_z: -0.05\r\n'
Time:5492293,Pitch:5.94,Yaw:88.17,Roll:-177.63,Accel_x: 0.72,Accel_y: 0.69,Accel_z: -0.05,Gyro_x: 0.72,Gyro_y: 0.69,Gyro_z: 


got b'Time:5492449,Pitch:5.94,Yaw:88.17,Roll:-177.63,Accel_x: 0.72,Accel_y: 0.69,Accel_z: -0.05,Gyro_x: 0.72,Gyro_y: 0.69,Gyro_z: -0.05,Mag_x: 0.72,Magl_y: 0.69,Mag_z: -0.05\r\n'
Time:5492449,Pitch:5.94,Yaw:88.17,Roll:-177.63,Accel_x: 0.72,Accel_y: 0.69,Accel_z: -0.05,Gyro_x: 0.72,Gyro_y: 0.69,Gyro_z: -0.05,Mag_x: 0.72,Magl_y: 0.69,Mag_z: -0.05

got b'Time:5492456,Pitch:5.94,Yaw:88.17,Roll:-177.63,Accel_x: 0.72,Accel_y: 0.69,Accel_z: -0.05,Gyro_x: 0.72,Gyro_y: 0.69,Gyro_z: -0.05,Mag_x: 0.72,Magl_y: 0.69,Mag_z: -0.05\r\n'
Time:5492456,Pitch:5.94,Yaw:88.17,Roll:-177.63,Accel_x: 0.72,Accel_y: 0.69,Accel_z: -0.05,Gyro_x: 0.72,Gyro_y: 0.69,Gyro_z: -0.05,Mag_x: 0.72,Magl_y: 0.69,Mag_z: -0.05

got b'Time:5492464,Pitch:5.94,Yaw:88.17,Roll:-177.63,Accel_x: 0.72,Accel_y: 0.69,Accel_z: -0.05,Gyro_x: 0.72,Gyro_y: 0.69,Gyro_z: -0.05,Mag_x: 0.72,Magl_y: 0.69,Mag_z: -0.05\r\n'
Time:5492464,Pitch:5.94,Yaw:88.17,Roll:-177.63,Accel_x: 0.72,Accel_y: 0.69,Accel_z: -0.05,Gyro_x: 0.72,Gyro_y: 0.69,Gyro_z: 

In [23]:
stride_process.start()

stride_processor starting...
decoded:  {',Gyro_x': '0.72', 'Gyro_y': '0.69', 'Gyro_z': '-0.05', 'Mag_x': '0.72', 'Magl_y': '0.69'}
decoded:  {'Time': '5491612', 'Pitch': '5.94', 'Yaw': '88.17', 'Roll': '-177.63', 'Accel_x': '0.72', 'Accel_y': '0.69', 'Accel_z': '-0.05', 'Gyro_x': '0.72', 'Gyro_y': '0.69', 'Gyro_z': '-0.05', 'Mag_x': '0.72', 'Magl_y': '0.69'}
decoded:  {'Time': '5491619', 'Pitch': '5.94', 'Yaw': '88.17', 'Roll': '-177.63', 'Accel_x': '0.72', 'Accel_y': '0.69', 'Accel_z': '-0.05', 'Gyro_x': '0.72', 'Gyro_y': '0.69', 'Gyro_z': '-0.05', 'Mag_x': '0.72', 'Magl_y': '0.69'}
decoded:  {'Time': '5491626', 'Pitch': '5.94', 'Yaw': '88.17', 'Roll': '-177.63', 'Accel_x': '0.72', 'Accel_y': '0.69', 'Accel_z': '-0.05', 'Gyro_x': '0.72', 'Gyro_y': '0.69', 'Gyro_z': '-0.05', 'Mag_x': '0.72', 'Magl_y': '0.69'}
decoded:  {'Time': '5491632', 'Pitch': '5.94', 'Yaw': '88.17', 'Roll': '-177.63', 'Accel_x': '0.72', 'Accel_y': '0.69', 'Accel_z': '-0.05', 'Gyro_x': '0.72', 'Gyro_y': '0.69', 'Gy

decoded:  {'Time': '5491853', 'Pitch': '5.94', 'Yaw': '88.17', 'Roll': '-177.63', 'Accel_x': '0.72', 'Accel_y': '0.69', 'Accel_z': '-0.05', 'Gyro_x': '0.72', 'Gyro_y': '0.69', 'Gyro_z': '-0.05', 'Mag_x': '0.72', 'Magl_y': '0.69'}
decoded:  {'Time': '5491861', 'Pitch': '5.94', 'Yaw': '88.17', 'Roll': '-177.63', 'Accel_x': '0.72', 'Accel_y': '0.69', 'Accel_z': '-0.05', 'Gyro_x': '0.72', 'Gyro_y': '0.69', 'Gyro_z': '-0.05', 'Mag_x': '0.72', 'Magl_y': '0.69'}
decoded:  {'Time': '5491867', 'Pitch': '5.94', 'Yaw': '88.17', 'Roll': '-177.63', 'Accel_x': '0.72', 'Accel_y': '0.69', 'Accel_z': '-0.05', 'Gyro_x': '0.72', 'Gyro_y': '0.69', 'Gyro_z': '-0.05', 'Mag_x': '0.72', 'Magl_y': '0.69'}
decoded:  {'Time': '5491873', 'Pitch': '5.94', 'Yaw': '88.17', 'Roll': '-177.63', 'Accel_x': '0.72', 'Accel_y': '0.69', 'Accel_z': '-0.05', 'Gyro_x': '0.72', 'Gyro_y': '0.69', 'Gyro_z': '-0.05', 'Mag_x': '0.72', 'Magl_y': '0.69'}
decoded:  {'Time': '5491881', 'Pitch': '5.94', 'Yaw': '88.17', 'Roll': '-177.63'

decoded:  {'Time': '5492092', 'Pitch': '5.94', 'Yaw': '88.17', 'Roll': '-177.63', 'Accel_x': '0.72', 'Accel_y': '0.69', 'Accel_z': '-0.05', 'Gyro_x': '0.72', 'Gyro_y': '0.69', 'Gyro_z': '-0.05', 'Mag_x': '0.72', 'Magl_y': '0.69'}
finished writing this one stride, size:  6
decoded:  {'Time': '5492100', 'Pitch': '5.94', 'Yaw': '88.17', 'Roll': '-177.63', 'Accel_x': '0.72', 'Accel_y': '0.69', 'Accel_z': '-0.05', 'Gyro_x': '0.72', 'Gyro_y': '0.69', 'Gyro_z': '-0.05', 'Mag_x': '0.72', 'Magl_y': '0.69'}
decoded:  {'Time': '5492106', 'Pitch': '5.94', 'Yaw': '88.17', 'Roll': '-177.63', 'Accel_x': '0.72', 'Accel_y': '0.69', 'Accel_z': '-0.05', 'Gyro_x': '0.72', 'Gyro_y': '0.69', 'Gyro_z': '-0.05', 'Mag_x': '0.72', 'Magl_y': '0.69'}
decoded:  {'Time': '5492112', 'Pitch': '5.94', 'Yaw': '88.17', 'Roll': '-177.63', 'Accel_x': '0.72', 'Accel_y': '0.69', 'Accel_z': '-0.05', 'Gyro_x': '0.72', 'Gyro_y': '0.69', 'Gyro_z': '-0.05', 'Mag_x': '0.72', 'Magl_y': '0.69'}
decoded:  {'Time': '5492120', 'Pitch'

decoded:  {'Time': '5492329', 'Pitch': '5.94', 'Yaw': '88.17', 'Roll': '-177.63', 'Accel_x': '0.72', 'Accel_y': '0.69', 'Accel_z': '-0.05', 'Gyro_x': '0.72', 'Gyro_y': '0.69', 'Gyro_z': '-0.05', 'Mag_x': '0.72', 'Magl_y': '0.69'}
decoded:  {'Time': '5492336', 'Pitch': '5.94', 'Yaw': '88.17', 'Roll': '-177.63', 'Accel_x': '0.72', 'Accel_y': '0.69', 'Accel_z': '-0.05', 'Gyro_x': '0.72', 'Gyro_y': '0.69', 'Gyro_z': '-0.05', 'Mag_x': '0.72', 'Magl_y': '0.69'}
finished writing this one stride, size:  6
decoded:  {'Time': '5492343', 'Pitch': '5.94', 'Yaw': '88.17', 'Roll': '-177.63', 'Accel_x': '0.72', 'Accel_y': '0.69', 'Accel_z': '-0.05', 'Gyro_x': '0.72', 'Gyro_y': '0.69', 'Gyro_z': '-0.05', 'Mag_x': '0.72', 'Magl_y': '0.69'}
decoded:  {'Time': '5492351', 'Pitch': '5.94', 'Yaw': '88.17', 'Roll': '-177.63', 'Accel_x': '0.72', 'Accel_y': '0.69', 'Accel_z': '-0.05', 'Gyro_x': '0.72', 'Gyro_y': '0.69', 'Gyro_z': '-0.05', 'Mag_x': '0.72', 'Magl_y': '0.69'}
decoded:  {'Time': '5492358', 'Pitch'

In [24]:
# arduino_proc.join()
# stride_process.join()
arduino_proc.terminate()
stride_process.terminate()

In [ ]:
x = 1

In [25]:
total_data

In [26]:
total_data.empty()

False

In [29]:
stride_1 = total_data.get()

In [30]:
stride_1

[{',Gyro_x': '0.72',
  'Gyro_y': '0.69',
  'Gyro_z': '-0.05',
  'Mag_x': '0.72',
  'Magl_y': '0.69'},
 {'Time': '5491612',
  'Pitch': '5.94',
  'Yaw': '88.17',
  'Roll': '-177.63',
  'Accel_x': '0.72',
  'Accel_y': '0.69',
  'Accel_z': '-0.05',
  'Gyro_x': '0.72',
  'Gyro_y': '0.69',
  'Gyro_z': '-0.05',
  'Mag_x': '0.72',
  'Magl_y': '0.69'},
 {'Time': '5491619',
  'Pitch': '5.94',
  'Yaw': '88.17',
  'Roll': '-177.63',
  'Accel_x': '0.72',
  'Accel_y': '0.69',
  'Accel_z': '-0.05',
  'Gyro_x': '0.72',
  'Gyro_y': '0.69',
  'Gyro_z': '-0.05',
  'Mag_x': '0.72',
  'Magl_y': '0.69'},
 {'Time': '5491626',
  'Pitch': '5.94',
  'Yaw': '88.17',
  'Roll': '-177.63',
  'Accel_x': '0.72',
  'Accel_y': '0.69',
  'Accel_z': '-0.05',
  'Gyro_x': '0.72',
  'Gyro_y': '0.69',
  'Gyro_z': '-0.05',
  'Mag_x': '0.72',
  'Magl_y': '0.69'},
 {'Time': '5491632',
  'Pitch': '5.94',
  'Yaw': '88.17',
  'Roll': '-177.63',
  'Accel_x': '0.72',
  'Accel_y': '0.69',
  'Accel_z': '-0.05',
  'Gyro_x': '0.72',
  '

In [ ]:
ard_pid

In [ ]:
os.kill(ard_pid, signal.SIGTERM)

In [ ]:
# start
# arduino_thread.start()
# stride_thread.start()
# arduino_process.start()


In [ ]:
# stride_process.start()

In [ ]:
# end
# arduino_thread.join()
# stride_thread.join()
# arduino_process.terminate()
# stride_process.terminate()
pool.close()
pool.join()

In [ ]:
manager

In [27]:
usb_reading_queue

In [ ]:
usb_reading_queue

In [ ]:
usb_reading_queue.put("hi")

In [28]:
usb_reading_queue.empty()

True

In [ ]:
x = usb_reading_queue.get()

In [ ]:
x

In [ ]:
total_data

In [ ]:
total_data.empty()

In [ ]:
strideX = total_data.get()

In [ ]:
strideX

In [ ]:
usb_reading_queue.qsize()

In [ ]:
# ///scratch

In [ ]:
usb_reading_queue = Queue()
while True:
    try:
        ser_bytes = ser.readline()
        try:
            print("got: ", ser_bytes)
            decoded_bytes = ser_bytes.decode('ascii')
            row = process_serial_out(decoded_bytes)
            print(decoded_bytes)
            usb_reading_queue.put(row)
        except:
            print_exc()
            print("exception reading one line from arduino")
            continue
    except:
        print_exc()
        print("Keyboard Interrupt")
        break

In [ ]:
usb_reading_queue.put(1)
usb_reading_queue.get()

In [ ]:
ser_bytes = b'time=6384885,accel_x= 0.00,accel_y= -0.02,accel_z= 0.00,gyro_x= -0.00,gyro_y= -0.00,gyro_z= 0.00,mag_x= 20.69,mag_y= 11.00,mag_z= -47.19,orientation_x= 300.62,orientation_y= 4.50,orientation_z= -2.06\r\n'
decoded_bytes = ser_bytes.decode('ascii')
row = process_serial_out(decoded_bytes)
print(decoded_bytes)
usb_reading_queue.put(row)



In [ ]:
usb_reading_queue.

In [ ]:
sdf = (source.map(pd.read_csv)                  # stream of Pandas dataframes
             .to_dataframe(example=...))        # logical streaming dataframe


In [ ]:

this_stride = []
full_data = []
data_chunk = []
while True:
#     if [chu]
    try:
        ser_bytes = ser.readline()
        try:
            decoded_bytes = ser_bytes.decode('ascii')
            row = process_serial_out(decoded_bytes)
            print(decoded_bytes)
            data_chunk.append(row)
        except:
            print("exception?")
            continue
#         with open("test_data.csv","a") as f:
#             writer = csv.writer(f,delimiter=",")
#             writer.writerow([time.time(),decoded_bytes])
#         y_var = np.append(y_var,decoded_bytes)
#         y_var = y_var[1:plot_window+1]
#         line.set_ydata(y_var)
#         ax.relim()
#         ax.autoscale_view()
#         fig.canvas.draw()
#         fig.canvas.flush_events()
    except:
        print("Keyboard Interrupt")
        break

In [ ]:
dummy = {'Time':'5948', 'Pitch': '3.65',
 'Yaw': '-119.85',
 'Roll': '179.54',
 'Accel_x': '-0.50',
 'Accel_y': '0.86',
 'Accel_z': '0.02',
 'Gyro_x': '-0.50',
 'Gyro_y': '0.86',
 'Gyro_z': '0.02',
 'Mag_x': '-0.50',
 'Magl_y': '0.86',
 'Mag_z': '0.02'}

In [ ]:
pd.DataFrame([dummy, dummy])

In [ ]:
df = pd.DataFrame(dummy, columns = [
    'Yaw'
,'Roll'
,'Accel_x'
,'Accel_y'
,'Accel_z'
,'Gyro_x'
,'Gyro_y'
,'Gyro_z'
,'Mag_x' 
,'Magl_y'
,'Mag_z'])


In [ ]:
pd.DataFrame.from_dict(dummy, orient='columns', dtype=None,typ='series')


In [ ]:
# serialPort = serial.Serial(port = portAddr, baudrate=115200, bytesize=8, timeout=2, stopbits=serial.STOPBITS_ONE)

In [ ]:
with open('hexdatafile.dat', 'wb') as datafile:
    datafile.write(ser.read(10000))

In [ ]:
msg = "Pitch:3.65,Yaw:-119.85,Roll:179.54,Accel_x: -0.50,Accel_y: 0.86,Accel_z: 0.02,Gyro_x: -0.50,Gyro_y: 0.86,Gyro_z: 0.02,Mag_x: -0.50,Magl_y: 0.86,Mag_z: 0.02"
msg += ","

In [ ]:
findall = re.findall(r'(?P<var>.*?):(?P<out>.*?),', msg)

In [ ]:
extracted = {k.strip(): v.strip() for k,v in findall}

In [ ]:
extracted